## HW 2 Solutions

### Import Libraries

In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from math import *
from numpy.linalg import inv

### Define Functions

In [3]:
def eta(P1, P2, xi):
    '''Function used in the Halpin-Tsai relationships.'''
    return ((P1/P2)-1)/((P1/P2)+xi)

def transIsoStiffness(E1, nu1, E2, nu2, V, HT=True):
    '''Return the 2D stiffness tensor of a transversely isotropic composite made
       from materials 1 and 2. Inputs required are the stiffness and Poisson's ratio of
       both materials (E1, nu1, E2, nu2) and volume fraction V of material 1.
       Properties are calculated using Halpin-Tsai relationships by default (HT = True),
       but a Reuss model can be used if desired.'''

    # Calculate Base Material Properties
    G1 = E1/(2*(1+nu1))
    G2 = E2/(2*(1+nu2))

    # Calculate Material Properties
    Ex = E1*V + E2*(1-V) #Voigt

    if HT: #Halpin-Tsai
        Ey = E2*(1 + 2*eta(E1,E2,2)*V)/(1 - eta(E1,E2,2)*V)
        Gxy = G2*(1 + 1*eta(G1,G2,1)*V)/(1 - eta(G1,G2,1)*V)
    else: #Reuss
        Ey = 1/(V/E1+(1-V)/E2)
        Gxy = 1/(V/G1+(1-V)/G2)

    nuxy = nu1*V+nu2*(1-V) #Voigt
    nuyx = nuxy*Ey/Ex #Reciprocal

    # Create Stiffness Matrix
    C = [[Ex/(1-nuxy*nuyx),      nuyx*Ex/(1-nuxy*nuyx), 0],
         [nuyx*Ex/(1-nuxy*nuyx), Ey/(1-nuxy*nuyx),      0],
         [0,                     0,                     Gxy]]

    return np.array(C)

def transIsoCompliance(E1, nu1, E2, nu2, V, HT=True):
    return inv(transIsoStiffness(E1, nu1, E2, nu2, V, HT=True))

def rotaT(theta):
    '''Matrix for stress rotation in 2D.'''
    c = cos(theta)
    s = sin(theta)
    return np.array([[c**2, s**2, 2*s*c],[s**2, c**2, -2*s*c],[-s*c, s*c, c**2-s**2]])

def rotaTP(theta):
    '''Matrix for strain rotation in 2D.'''
    c = cos(theta)
    s = sin(theta)
    return np.array([[c**2, s**2, s*c],[s**2, c**2, -s*c],[-2*s*c, 2*s*c, c**2-s**2]])

def rotateStress(stress, theta):
    '''Rotate 2D stress counterclockwise. Input stress as a 3x1 vector'''
    T = rotaT(theta)
    return T.dot(stress)

def rotateStrain(strain, theta):
    '''Rotate 2D strain counterclockwise. Input  strain as a 3x1 vector'''
    TP = rotaTP(theta)
    return TP.dot(strain)

def rotateStiffness(C, theta):
    '''Rotate the 2D stiffness tensor C counterclockwise by angle theta.'''
    # Define rotation matrix T and T'
    T = rotaT(theta)
    TP = rotaTP(theta)

    # Stress tensor rotations
    return inv(T).dot(C).dot(TP)

def rotateCompliance(S, theta):
    '''Rotate the 2D compliance tensor S counterclockwise by angle theta.'''
    # Define rotation matrix T and T'
    T = rotaT(theta)
    TP = rotaTP(theta)

    # Stress tensor rotations
    return inv(TP).dot(S).dot(T)

def laminateStiffness(C, thetas, ts):
    '''Return the stiffness of a laminate given a single input C aligned with the
    fiber orientation of the base material, an array of angles corresponding to the
    laminate stack and an array of thicknesses ts corresponding to each lamina.'''
    return sum([rotateStiffness(C,theta)*t for theta,t in zip(thetas,ts)])/sum(ts)

def laminateCompliance(CLam):
    return inv(CLam)

def shortfiberstiffness(Em, Ef, V, s, vm):
    '''Calculates the stiffness of a short fiber composite using the modified shear lag model.
    Inputs are stiffness of fiber and matrix (Ef and Em), Volume fraction of fiber, fiber
    aspect ratio (s) and matrix Poisson's ratio vm'''
    n = np.sqrt(2*Em/(Ef*(1+vm)*np.log(1/V)))
    a = np.cosh(n*s)
    Emprime = (Ef*(1-(1/a)) + Em)
    angle = n*s*180/(np.pi)
    E = V*Ef*(1-((Ef-Emprime)*np.tanh(n*s))/(Ef*n*s)) + (1-V)*Em
    return E

def isLaminateBalanced(S, plot):
    '''Determine whether a laminate is balanced by inputting a compliance matrix S.
    Plot is True or False and will plot the tension-shear interaction ratios'''

    # Define angles between 0-90 degrees
    ths = [x*pi/180 for x in range(91)]

    # Find tension-shear interaction ratios
    nxyx = []; nxyy = []
    for th in ths:
      SLR = rotateCompliance(S, th)
      nxyx += [SLR[0][2]*SLR[0][0]]
      nxyy += [SLR[1][2]*SLR[1][1]]

    # Plot the results
    if plot:
        fig,axs = plt.subplots(nrows=2, ncols=1)
        axs[0].plot(ths, nxyx)
        axs[1].plot(ths, nxyy)
        axs[1].set_xlabel(r'Angle $\theta$')
        axs[0].set_ylabel(r'$\eta_{xyx}$')
        axs[1].set_ylabel(r'$\eta_{xyy}$')

    # Return balanced or not
    return all(x==0 for x in nxyx) and all(x==0 for x in nxyy)

def Thermal_strain(e1,e2,g12, alphm, alphf, Em, Ef, vm, vf, f, delT):
    '''Calculates the thermal strain induced in a composite'''
    alph11 = (alphf*f*Ef + alphm*(1-f)*Em)/(f*Ef + (1-f)*Em)
    alph22 = alphf*f*(1+vf) + alphm*(1-f)*(1+vm) - alph11*(f*vf + (1-f)*vm)
    strain = np.zeros((3,1))
    strain[0,0] = e1-alph11*delT
    strain[1,0] = e2-alph22*delT
    strain[2,0] = g12
#    print(alph11, alph22)
    return strain

### Q3

In [4]:
C_3 = transIsoStiffness(320,0.21,2.4,0.42,0.65, HT=False) #GPa
e_3 = Thermal_strain(0,0,0, 90*10**(-6), 0.3*10**(-6), 2.4, 320, 0.42, 0.21, 0.65, 80-20)
stress_3= C_3.dot(e_3) #GPa
# print("Sigma=",stress_3, "GPa")

### Q4

In [5]:
C_input_4 = ([[46500,2100,0],[2100,6000,0], [0,0,1600]]) #MPa
stress_global_4 = ([20, -5, 5]) #MPa
theta_4 = ([0, np.pi/2])
ts_4 = ([1,1])
C_global_4 = laminateStiffness(C_input_4, theta_4, ts_4)
S_global_4 = laminateCompliance(C_global_4)
strain_global_4 = S_global_4.dot(stress_global_4)
stress_local_0_4 = np.matmul(C_input_4, strain_global_4)
# print("Stress=", stress_local_0_4, "MPa")

### Q5

In [6]:
C1_5 = np.array([[91.05, 30.59,  0], [30.59, 91.05, 0], [0,0,30.23]])
C2_5 = np.array([[115.85, 5.79,0], [5.79, 115.85, 0], [0, 0, 5.42]])
S1_5 = np.linalg.inv(C1_5)
S2_5 = np.linalg.inv(C2_5) 

# Laminate_1_5 = isLaminateBalanced(S1_5, True)
# Laminate_2_5 = isLaminateBalanced(S2_5, True)

### Q6

In [7]:
# Input Properties
Em = 72     #GPa
Ef = 192    #Gpa
V = 0.35
vm = 0.33
vf = 0.16
s = np.linspace(1,10,10)

# Calculation for Modified Shear Lag model
Modshearlag = []
for i in s:
    answer = shortfiberstiffness(Em, Ef, V, s, vm)
    Modshearlag.append(answer)

# Rule of Mixtures
E2 = V*Ef + (1-V)*Em

# MSL_plot = plt.plot(s,Modshearlag[0], 'k--', label='Modified Shear lag Model')
# MSL_plot = plt.axhline(y = E2, linestyle=':',label='Rule of Mixtures')
# plt.xlabel =('Reinforcement aspect ratio')
# plt.ylabel = ("Modulus (GPa)")
# legend = plt.legend(loc='lower right', fontsize='medium')

### Q7

In [8]:
C_input_7 = ([151, 3.4, 0], [3.4,16.7,0], [0,0,4.7])
theta_7 = ([0, np.pi/2, np.pi/3, -np.pi/3, np.pi/6, -np.pi/6 ])
ts_7 =  ([1,1,0.5,0.5,0.5,0.5])

Answer_7 = laminateStiffness(C_input_7, theta_7, ts_7)
# print(Answer_7)

### Q8

In [9]:
strain_8 = ([600*10**-6, 300*10**-6, -120*10**-6])
theta_8 = np.arange(0, np.pi/2, np.pi/18)

C_90_8 = rotateStiffness(C_input_7, np.pi/2)
strain_90_8 = rotateStrain(strain_8, np.pi/2)
stress_90_8 = C_90_8.dot(strain_90_8)
Answer_8 = []
for i in theta_8:
   stress = rotateStress(stress_90_8, i)
   Answer_8.append(stress[0])

# axialstress_plot = plt.plot(theta_8,Answer_8, 'k--', label='Rotated Axial Stress')
# plt.xlabel("Theta")
# plt.ylabel("Stress")